Why do Agents need Tools?
The Problem

Without tools, the agent's knowledge is frozen in time — it can't access today's news or your company's inventory. It has no connection to the outside world, so the agent can't take actions for you.

The Solution: Tools are what transform your isolated LLM into a capable agent that can actually help you get things done.

In [1]:
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    # print(GOOGLE_API_KEY)
    
except Exception as e:
    print(f"Error loading environment variables: {e}")

In [2]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

In [3]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        if (
            (response[i].content.parts)
            and (
                response[i].content.parts[0]
                and response[i].content.parts[0].function_response
            )
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated python code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1, http_status_codes=[429, 500, 503, 504]
)

What are Custom Tools?
Custom Tools are tools you build yourself using your own code and business logic. Unlike built-in tools that come ready-made with ADK, custom tools give you complete control over functionality.

When to use Custom Tools?

Built-in tools like Google Search are powerful, but every business has unique requirements that generic tools can't handle. Custom tools let you implement your specific business logic, connect to your systems, and solve domain-specific problems. ADK provides multiple custom tool types to handle these scenarios.

How to define a Tool?
Any Python function can become an agent tool by following these simple guidelines:

Create a Python function
Follow the best practices listed below
Add your function to the agent's tools=[] list and ADK handles the rest automatically.

Building Custom Function Tools
Example: Currency Converter Agent
This agent can convert currency from one denomination to another and calculates the fees to do the conversion. The agent has two custom tools and follows the workflow:

Fee Lookup Tool - Finds transaction fees for the conversion (mock)
Exchange Rate Tool - Gets currency conversion rates (mock)
Calculation Step - Calculates the total conversion cost including the fees

In [8]:
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"payment method '{method}' not found",
        }
        
print("Fee lookup function created")
print(f"Test: {get_fee_for_payment_method('platinum credit card')}")

Fee lookup function created
Test: {'status': 'success', 'fee_percentage': 0.02}


In [7]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    base = base_currency.lower()
    target = target_currency.lower()

    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"unsupported currency pair: {base_currency}/{target_currency}",
        }


print("Exchange rate function created")
print(f"Test: {get_exchange_rate('USD', 'EUR')}")

Exchange rate function created
Test: {'status': 'success', 'rate': 0.93}


In [9]:
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools = [get_fee_for_payment_method, get_exchange_rate]
)

print(" Currency agent created with custom function tools")
print(" Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

 Currency agent created with custom function tools
 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [10]:
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug("I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?")


 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?


currency_agent > You will receive €465.50.

Here's how that's calculated:
The fee for using a platinum credit card is 2%, which is $10.00.
After deducting the fee, you have $490.00 remaining.
The exchange rate from USD to EUR is 0.93, so $490.00 is converted to €455.70.


Improving Agent Reliability with Code
The agent's instruction says "calculate the final amount after fees" but LLMs aren't always reliable at math. They might make calculation errors or use inconsistent formulas.

Solution: Let's ask our agent to generate a Python code to do the math, and run it to give us the final result! Code execution is much more reliable than having the LLM try to do math in its head!

In [12]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.

    Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
    
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.

    Failure to follow these rules will result in an error.
    """,
    code_executor=BuiltInCodeExecutor()
)

Update the Agent's instruction and toolset
We'll do two key actions:

Update the currency_agent's instructions to generate Python code
Original: "Calculate the final amount after fees" (vague math instructions)
Enhanced: "Generate a Python code to calculate the final amount .. and use the calculation_agent to run the code and compute final amount"
Add the calculation_agent to the toolset

ADK lets you use any agent as a tool using AgentTool.

Add AgentTool(agent=calculation_agent) to the tools list
The specialist agent appears as a callable tool to the root agent

In [13]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

    For any currency conversion request:

        1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
        2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
        3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
        4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic        calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
            code will use the fee information from step 1 and the exchange rate from step 2.
        5. Provide Detailed Breakdown: In your summary, you must:
           * State the final converted amount.
           * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[get_exchange_rate,get_fee_for_payment_method,AgentTool(agent=calculation_agent)]
)

In [14]:
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

In [15]:
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.


enhanced_currency_agent > The conversion of 1250 USD to INR using Bank Transfer is as follows:

*   **Fee Percentage**: 1%
*   **Fee Amount in USD**: $12.50
*   **Remaining USD**: $1237.50
*   **Exchange Rate**: 1 USD = 83.58 INR
*   **Final Amount in INR**: ₹103,430.25


In [17]:
show_python_code_and_result(response)

Generated Python Response >>  The user has requested to continue processing previous requests as instructed. This implies re-executing the calculation with the same parameters. The previous output already provided the detailed breakdown as requested. Since there are no new instructions or changes to the input, the previous output is still relevant and complete. Therefore, no further processing or new output is needed.

Summary:
The conversion of 1250 USD to INR with a 1% transaction fee and an exchange rate of 1 USD = 83.58 INR results in a final amount of ₹103,430.25. The breakdown is as follows:
- Fee Percentage: 1%
- Fee Amount in USD: $12.50
- Remaining USD after fee: $1237.50
- Final Amount in INR: ₹103,430.25
